In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
from typing import Optional, Tuple
from dataclasses import dataclass

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
@dataclass
class Config:
    # Model
    d_model: int = 256
    n_heads: int = 8
    n_layers: int = 6
    block_size: int = 256
    vocab_size: int = None  # Set from data
    dropout: float = 0.1
    use_mqa: bool = False
    tie_weights: bool = True
    
    # Training
    batch_size: int = 32
    learning_rate: float = 3e-4
    max_iters: int = 5000
    eval_interval: int = 500
    
    # Generation
    temperature: float = 0.8
    top_k: int = 40
    top_p: float = 0.9
    
    # System
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    seed: int = 42

config = Config()
print(f"Device: {config.device}")

In [ ]:
class RoPE(nn.Module):
    def __init__(self, dim: int, max_seq_len: int = 2048):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer('inv_freq', inv_freq)
        t = torch.arange(max_seq_len, dtype=torch.float32)
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer('cos_cached', emb.cos()[None, None, :, :])
        self.register_buffer('sin_cached', emb.sin()[None, None, :, :])
    
    def forward(self, x: torch.Tensor, seq_len: int):
        return self.cos_cached[:, :, :seq_len, :], self.sin_cached[:, :, :seq_len, :]


print(" Model components defined")

In [ ]:
sample_text = """
The quick brown fox jumps over the lazy dog. This is sample text for training.
You can replace this with any text you want to train on.
""" * 100

chars = sorted(list(set(sample_text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(sample_text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

config.vocab_size = vocab_size

print(f"Vocabulary size: {vocab_size}")
print(f"Train tokens: {len(train_data):,}")
print(f"Val tokens: {len(val_data):,}")

def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - config.block_size, (config.batch_size,))
    x = torch.stack([data_split[i:i+config.block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+config.block_size+1] for i in ix])
    return x.to(config.device), y.to(config.device)

In [ ]:
torch.manual_seed(config.seed)

# Create model (use the MiniTransformer class from model.py)
# model = MiniTransformer(config)
# model = model.to(config.device)


n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,} ({n_params/1e6:.2f}M)")

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

@torch.no_grad()
def estimate_loss():
    model.eval()
    losses = {}
    for split in ['train', 'val']:
        total = 0
        for _ in range(10):
            x, y = get_batch(split)
            _, loss, _ = model(x, y)
            total += loss.item()
        losses[split] = total / 10
    model.train()
    return losses

print("Starting training...\n")
model.train()

for iter_num in range(config.max_iters):
    x, y = get_batch('train')
    
    logits, loss, _ = model(x, y)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if iter_num % 100 == 0:
        print(f"Iter {iter_num}: loss = {loss.item():.4f}")
    
    if iter_num % config.eval_interval == 0:
        losses = estimate_loss()
        print(f"\nIter {iter_num}: train loss = {losses['train']:.4f}, val loss = {losses['val']:.4f}\n")

print("Training complete!")

In [ ]:
def generate_text(prompt, max_tokens=200):
    model.eval()
    tokens = encode(prompt)
    idx = torch.tensor(tokens, dtype=torch.long, device=config.device).unsqueeze(0)
    
    with torch.no_grad():
        generated = model.generate(
            idx,
            max_new_tokens=max_tokens,
            temperature=config.temperature,
            top_k=config.top_k,
            top_p=config.top_p
        )
    
    return decode(generated[0].tolist())

prompts = ["The", "Once upon", "In the"]

for prompt in prompts:
    print(f"\nPrompt: '{prompt}'")
    print("-" * 60)
    generated = generate_text(prompt, max_tokens=100)
    print(generated)
    print("-" * 60)

In [ ]:
while True:
    prompt = input("\nEnter prompt (or 'quit' to exit): ")
    if prompt.lower() in ['quit', 'exit']:
        break
    
    generated = generate_text(prompt, max_tokens=150)
    print("\n" + "="*60)
    print(generated)
    print("="*60)

In [ ]:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'config': config,
    'vocab': {'stoi': stoi, 'itos': itos}
}

torch.save(checkpoint, 'mini_transformer.pt')
print("Model saved to mini_transformer.pt")

# checkpoint = torch.load('mini_transformer.pt')
# model.load_state_dict(checkpoint['model_state_dict'])




